# Use a table pipeline as a reusable function

Convert a table with computed columns into a callable function for multi-agent workflows and pipeline composition.


## Problem

You have a table that runs a complex pipeline—LLM calls, tool use, post-processing—and you want to reuse that entire pipeline from other tables. Copy-pasting computed column definitions is error-prone and hard to maintain.

| Scenario | Pipeline | Consumers |
|----------|----------|-----------|
| Research agent | Web search → LLM analysis → Summary | Report generator, Dashboard |
| Data enrichment | Lookup → Transform → Validate | Orders table, Analytics |
| Content moderation | Vision analysis → Text check → Score | User uploads, Comments |


## Solution

**What's in this recipe:**
- Create an "agent" table with computed columns
- Convert the table to a callable UDF with `pxt.udf(table, return_value=...)`
- Use the table UDF in other tables' computed columns

You wrap an entire table pipeline as a function. When you call this function from another table, it inserts a row into the agent table, runs all computed columns, and returns the specified output column.


### Setup


In [1]:
%pip install -qU pixeltable openai


Note: you may need to restart the kernel to use updated packages.


In [ ]:
import os
import getpass

if 'OPENAI_API_KEY' not in os.environ:
    os.environ['OPENAI_API_KEY'] = getpass.getpass('OpenAI API Key: ')


In [ ]:
import pixeltable as pxt
from pixeltable.functions.openai import chat_completions


In [ ]:
# Create a fresh directory
pxt.drop_dir('table_udf_demo', force=True)
pxt.create_dir('table_udf_demo')


### Create an agent table with computed columns

You create a table that encapsulates a complete pipeline. This example builds a summarization agent:


In [ ]:
# Create the agent table with input column
summarizer = pxt.create_table(
    'table_udf_demo.summarizer',
    {'text': pxt.String}
)


In [ ]:
# Add the LLM call as a computed column
summarizer.add_computed_column(
    response=chat_completions(
        messages=[{
            'role': 'user',
            'content': 'Summarize this in one sentence:\n\n' + summarizer.text
        }],
        model='gpt-4o-mini'
    )
)


In [ ]:
# Extract the summary text
summarizer.add_computed_column(
    summary=summarizer.response.choices[0].message.content
)


### Convert the table to a UDF

You use `pxt.udf(table, return_value=...)` to convert the table into a callable function. The `return_value` specifies which column to return:


In [ ]:
# Convert the summarizer table into a callable UDF
summarize = pxt.udf(
    summarizer,
    return_value=summarizer.summary
)


### Use the table UDF in another table

You can now use `summarize()` as a computed column in any other table:


In [ ]:
# Create a table that uses the summarizer
articles = pxt.create_table(
    'table_udf_demo.articles',
    {'title': pxt.String, 'content': pxt.String}
)


In [ ]:
# Add the table UDF as a computed column
articles.add_computed_column(
    summary=summarize(text=articles.content)
)


In [ ]:
# Insert articles - summaries are generated automatically
articles.insert([
    {
        'title': 'Climate Report',
        'content': 'Global temperatures rose by 1.2 degrees Celsius above pre-industrial levels last year, marking the hottest year on record. Scientists attribute this to continued greenhouse gas emissions and a strong El Nino pattern. The report calls for immediate action to reduce carbon emissions.'
    },
    {
        'title': 'Tech Merger',
        'content': 'Two major semiconductor companies announced a merger valued at $50 billion. The combined entity will control 30% of the global chip market. Regulators in multiple countries will review the deal over the next 18 months.'
    },
])


In [ ]:
# View results
articles.select(articles.title, articles.summary).collect()


### Chain multiple table UDFs

You can compose table UDFs by using one's output as another's input:


In [ ]:
# Create a translator agent table
translator = pxt.create_table(
    'table_udf_demo.translator',
    {'text': pxt.String}
)

translator.add_computed_column(
    response=chat_completions(
        messages=[{
            'role': 'user',
            'content': 'Translate to Spanish:\n\n' + translator.text
        }],
        model='gpt-4o-mini'
    )
)

translator.add_computed_column(
    translation=translator.response.choices[0].message.content
)


In [ ]:
# Convert to UDF
translate = pxt.udf(
    translator,
    return_value=translator.translation
)


In [ ]:
# Chain: summarize first, then translate the summary
articles.add_computed_column(
    summary_es=translate(text=articles.summary)
)


In [ ]:
# View the chain results
articles.select(articles.title, articles.summary, articles.summary_es).collect()


## Explanation

**How table UDFs work:**

```
Consumer table row → Table UDF called → Agent table inserts row → 
Computed columns run → Return value extracted → Consumer gets result
```

**When to use table UDFs vs `@pxt.query`:**

| Pattern | Use when |
|---------|----------|
| Table UDF | You need to run a multi-step pipeline (LLM → process → LLM) |
| `@pxt.query` | You need to retrieve/search existing data |
| `@pxt.udf` | You need a pure Python transformation |

**Key benefits:**
- **Encapsulation**: Hide complex pipeline details behind a simple function call
- **Reusability**: Use the same agent from multiple consumer tables
- **Persistence**: All intermediate results are stored in the agent table for debugging
- **Composition**: Chain agents together for multi-stage workflows


## See also

- [Look up structured data](./pattern-data-lookup.ipynb) - Simple key-based lookups with `retrieval_udf`
- [Build a RAG pipeline](./pattern-rag-pipeline.ipynb) - Retrieval with `@pxt.query`
- [Use tool calling with LLMs](./llm-tool-calling.ipynb) - Add tools to agent tables
